In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import torch.nn.functional as F
import torch
import torch.nn as nn
import gc
import tqdm

In [ ]:
# cells code which is not used once again for training is commented

In [ ]:
# table image starts with "_"

# approach is to
# unzip archive 
# use pretrained net (small efficientNet v2)
# compute feature representations and store them in dataframe(for future csv representation)
# use approximate nearest neigbours approach index to store this data
# find some statistical dataframe parameters (number of photos N for every object in average, also min and max are important)
# this might allow to find N nearest neigbours fast using this approach
# after finding N nearest neigbours voting and classification determines the actual data

In [ ]:
# unzip

with zipfile.ZipFile('drive/MyDrive/data_for_tesk_task/001-Pompidou.zip', 'r') as zip_ref:
    zip_ref.extractall('data/')

In [ ]:

PROJECT_FOLDER_PATH = 'drive/MyDrive/data_for_tesk_task'

IMAGES_PATH = 'data/'

TABLE_IMAGES_START_SYMBOL = '_'


In [ ]:
#creating dataframe with images_paths

data = []

# current structure is (class(folderName),imagepath)

for class_folder in os.listdir(IMAGES_PATH):
  complete_path_class_folder = os.path.join(IMAGES_PATH,class_folder)
  if os.path.isdir(complete_path_class_folder):
    for class_folder_file in os.listdir(complete_path_class_folder):
      if class_folder_file[0]!='_':
        data.append((class_folder,class_folder_file))
  
df = pd.DataFrame(data,columns=['class','image'])

In [ ]:
# df.sample(5)

In [ ]:
# df.shape

In [ ]:
# df.groupby('class').count().describe()

In [ ]:
# according to current statisic would be ok to take 9 photos for ANN algorithm
# cuz of having 5 photos minimal would be enough to classify and image using
# majority vote rule
# actually it is recomended to have more photos and also experiment with different features
# but as for test task it would be totally ok i suppose to use a simple working algorithm

In [ ]:
# df.to_csv(os.path.join(PROJECT_FOLDER_PATH,'images.csv'),index=False)
df = pd.read_csv(os.path.join(PROJECT_FOLDER_PATH,'images.csv'))

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
torch.cuda.empty_cache()

In [ ]:
# the next part is to create feature representation vector for every immage using pretrained net
# firstly create,upload and test model on CPU, than run bactch GPU script


# object, wrapper or apply func to store array in df cell
import timm
timm.list_models('*efficientnetv2_s*',pretrained=True)

['tf_efficientnetv2_s',
 'tf_efficientnetv2_s_in21ft1k',
 'tf_efficientnetv2_s_in21k']

In [ ]:
from torchvision.models import efficientnet_v2_s

base_model = timm.create_model('tf_efficientnetv2_s',pretrained=True)
base_model.eval()


EfficientNet(
  (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
base_model.to(device)

EfficientNet(
  (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [ ]:
base_model.training

False

In [ ]:
o = base_model.forward_features(torch.randn(2, 3, 256, 256).to(device))

In [ ]:
o.shape

torch.Size([2, 1280, 8, 8])

In [ ]:
del o
gc.collect()

135

In [ ]:
import torch.nn as nn
import torch.optim as optim

class Network(nn.Module):
    def __init__(self, emb_dim=256):
        super(Network, self).__init__()
        self.base_model = base_model.forward_features
        
        self.fc = nn.Sequential(
            nn.Linear(1280*8*8, 2048),
            nn.PReLU(),
            nn.Linear(2048, emb_dim)
        )
        
    def forward(self, x):
        x = self.base_model(x)
        x = x.view(-1, 1280*8*8)
        x = self.fc(x)
        # x = nn.functional.normalize(x)
        return x

In [ ]:
model = Network(256)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.TripletMarginLoss(margin=1.0, p=2)

In [ ]:
df['path_restored'] = df['class'] + '/' + df['image']
df['path_full'] = df['path_restored'].apply(lambda p: os.path.join(IMAGES_PATH,p))
df.sample(3)

,class,image,path_restored,path_full
2513,Inferno Hallelujah,IMG_1935.HEIC,Inferno Hallelujah/IMG_1935.HEIC,data/Inferno Hallelujah/IMG_1935.HEIC
2621,Paysan au parapluie,"Фото 17.04.2023, 16 24 20.jpg","Paysan au parapluie/Фото 17.04.2023, 16 24 20.jpg","data/Paysan au parapluie/Фото 17.04.2023, 16 2..."
444,Le Monstre de Soisy,IMG_1493.HEIC,Le Monstre de Soisy/IMG_1493.HEIC,data/Le Monstre de Soisy/IMG_1493.HEIC


In [ ]:
!pip install pillow-heif

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 51.5 MB/s eta 0:00:00


In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from pillow_heif import register_heif_opener

register_heif_opener()


class TLDS(Dataset):
    def __init__(self, df, train=True, transform=None):
        self.is_train = train
        self.transform = transform
                
        self.image_paths = df['path_full']
        self.labels = df['class']
        self.index = df.index.values
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        anchor_img_path = self.image_paths[index]
        anchor_label = self.labels[index]
        if self.is_train:
            

            positive_list = self.index[self.index!=index][self.labels[self.index!=index]==anchor_label]
            positive_index = np.random.choice(positive_list)
            positive_img_path = self.image_paths[positive_index]
            positive_label = self.labels[positive_index]
            
            negative_list = self.index[self.index!=index][self.labels[self.index!=index]!=anchor_label]
            negative_index = np.random.choice(negative_list)
            negative_img_path = self.image_paths[negative_index]
            negative_label = self.labels[negative_index]

            anchor_img = Image.open(anchor_img_path)
            positive_img = Image.open(positive_img_path)
            negative_img = Image.open(negative_img_path)


            anchor_image_tensor = self.transform(anchor_img)
            positive_image_tensor = self.transform(positive_img)
            negative_image_tensor = self.transform(negative_img)
            
            return anchor_image_tensor, positive_image_tensor, negative_image_tensor, anchor_label,positive_label,negative_label
        
        else:
            anchor_img = self.transform(Image.open(anchor_img_path))
            return anchor_img,anchor_label


class CustomDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path)
        image_tensor = self.transform(image)
        return image_tensor

    def __len__(self):
        return len(self.image_paths)

transformations = transforms.Compose([
    transforms.Resize([256,256]),
    transforms.ToTensor(),
])

BATCH_SIZE = 16

train_ds = TLDS(df, 
                 train=True,
                 transform=transformations)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
gc.collect()

97

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
epochs = 3

In [ ]:
model.train()
torch.cuda.empty_cache()
for epoch in tqdm.tqdm(range(epochs), desc="Epochs"):
    running_loss = []
    for step, (anchor_img, positive_img, negative_img, anchor_label,positive_label,negative_label) in enumerate(tqdm.tqdm(train_loader, desc="Training", leave=False)):
        if step==2:
          break
        anchor_img = anchor_img.to(device)
        positive_img = positive_img.to(device)
        negative_img = negative_img.to(device)
        
        optimizer.zero_grad()
        anchor_out = model(anchor_img)
        positive_out = model(positive_img)
        negative_out = model(negative_img)
        
        loss = criterion(anchor_out, positive_out, negative_out)
        loss.backward()
        optimizer.step()

        del anchor_img,positive_img,negative_img
        gc.collect()
        torch.cuda.empty_cache()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))

Epochs:  33%|███▎      | 1/3 [01:35<03:10, 95.28s/it]

Epoch: 1/3 - Loss: 5.9618



Epochs:  33%|███▎      | 1/3 [01:39<03:18, 99.23s/it]


KeyboardInterrupt: ignored

In [ ]:
eval_ds = TLDS(df, 
                 train=False,
                 transform=transformations)
eval_loader = DataLoader(eval_ds, batch_size=BATCH_SIZE, shuffle=False)


feature_representations = []
labels = []
gc.collect()
torch.cuda.empty_cache()
model.eval()
model.cuda()
for img,label in tqdm.tqdm(eval_loader):
  img = img.to(device)
  with torch.no_grad():
    computed_representations = model(img).tolist()
  labels +=label
  feature_representations+=computed_representations
  del img
  torch.cuda.empty_cache()
  gc.collect()

100%|██████████| 207/207 [31:57<00:00,  9.26s/it]


In [ ]:
import pickle

DRIVE_PATH = 'drive/MyDrive/data_for_tesk_task/'

first_dump_path = os.path.join(DRIVE_PATH,'list_256_emb_dump_last.pkl')

with open(first_dump_path, 'wb') as f:
    pickle.dump(feature_representations, f)


second_dump_path = os.path.join(DRIVE_PATH,'labels_last.pkl')

with open(second_dump_path, 'wb') as f:
    pickle.dump(labels,f)


In [ ]:
torch.save(model.to('cpu').state_dict(), os.path.join(DRIVE_PATH,'model_last_state_dict_CPU.pth'))

In [ ]:
model.load_state_dict(torch.load(os.path.join(DRIVE_PATH,'model_last_state_dict.pth')))

<All keys matched successfully>

In [ ]:
model.eval()

Network(
  (fc): Sequential(
    (0): Linear(in_features=81920, out_features=2048, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Linear(in_features=2048, out_features=256, bias=True)
  )
)

In [ ]:
torch.save({"model_state_dict": model.state_dict(),
            "optimzier_state_dict": optimizer.state_dict()
           }, os.path.join(DRIVE_PATH,'model_256_emb_dump.pth'))

In [ ]:
del feature_representations
gc.collect()

112

In [ ]:
del computed_representations
gc.collect()

486

In [ ]:
df.iloc[26*64:]['path_full']

1664    drive/MyDrive/data_for_tesk_task/data/Bewtiful...
1665    drive/MyDrive/data_for_tesk_task/data/Bewtiful...
1666    drive/MyDrive/data_for_tesk_task/data/Bewtiful...
1667    drive/MyDrive/data_for_tesk_task/data/Bewtiful...
1668    drive/MyDrive/data_for_tesk_task/data/Bewtiful...
                              ...                        
3296    drive/MyDrive/data_for_tesk_task/data/Les Chif...
3297    drive/MyDrive/data_for_tesk_task/data/Les Chif...
3298    drive/MyDrive/data_for_tesk_task/data/Les Chif...
3299    drive/MyDrive/data_for_tesk_task/data/Les Chif...
3300    drive/MyDrive/data_for_tesk_task/data/Les Chif...
Name: path_full, Length: 1637, dtype: object

In [ ]:
dataset = CustomDataset(df.iloc[26*64:]['path_full'].values, transformations)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

feature_representations = []

torch.cuda.empty_cache()
model.eval()
model.cuda()

EfficientNet(
  (conv_stem): Conv2dSame(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
      (1): ConvBnAct(
        (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (drop_path): Identity()
      )
    )
    (1): Sequential(
   

In [ ]:
for batch in tqdm.tqdm(dataloader):
  batch = batch.to(device)
  with torch.no_grad():
    computed_representations = model.forward_features(batch).flatten(start_dim=1).tolist()
  feature_representations+=computed_representations
  del batch
  torch.cuda.empty_cache()
  gc.collect()

100%|██████████| 26/26 [25:21<00:00, 58.54s/it]


In [ ]:
del feature_representations
gc.collect()

8

In [ ]:
len(restored)

1637